In [1]:
import sys
from mainChatBot import ChatBot
from transformers.utils import logging

logging.set_verbosity_error() # shut down warnings from transformers

c:\Users\nibaaa\.conda\envs\dia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CHATBOT=ChatBot()

In [3]:
news = CHATBOT.get_latest_hot_news_with_summary()

# 输出结果
for i, itme in enumerate(news, 1):
    print(f"news {i}")
    print(f"title: {itme['title']}")

print("=======================")
first_news = news[0]
print(f"first news title: {first_news['title']}")
print(f"first news summary: {first_news['summary']}")

news 1
title: Top UK Special Forces general oversaw blocking of Afghan 'war-crime' witnesses to Britain
news 2
title: Police investigate fire at PM Starmer's house in north London
news 3
title: PM promises migration drop as he unveils plans for 'tightened' visa rules
first news title: Top UK Special Forces general oversaw blocking of Afghan 'war-crime' witnesses to Britain
first news summary: of the heart of best online games of how those who is <OOV> at its dvd format the ds who do not even to <OOV> in quality way to have an <OOV> of problems in the world years the <OOV> of 64 <OOV> 64 speed <OOV> davies alan <OOV> snap shot was charged down and dyer reacted first to smash the ball past the <OOV> <OOV> from six yards goals <OOV> 41 bolton were creating little going forward and they seemed content to <OOV> the magpies in the first half we had put them under a lot of pressure and our goal matched theirs in quality lee bowyer put newcastle ahead when he fed stephen carr on the game of th

In [4]:
first_news = news[0]
print(f"first news title: {first_news['title']}")
print(f"first news summary: {first_news['summary']}")

first news title: Top UK Special Forces general oversaw blocking of Afghan 'war-crime' witnesses to Britain
first news summary: of the heart of best online games of how those who is <OOV> at its dvd format the ds who do not even to <OOV> in quality way to have an <OOV> of problems in the world years the <OOV> of 64 <OOV> 64 speed <OOV> davies alan <OOV> snap shot was charged down and dyer reacted first to smash the ball past the <OOV> <OOV> from six yards goals <OOV> 41 bolton were creating little going forward and they seemed content to <OOV> the magpies in the first half we had put them under a lot of pressure and our goal matched theirs in quality lee bowyer put newcastle ahead when he fed stephen carr on the game of the <OOV> and <OOV> delays <OOV> safe to say that this is the most highly anticipated computer game of all time the bar has been raised and so far out of sight that to play the game will have an


In [5]:
from rouge_score import rouge_scorer
from bert_score import score as bert_score

In [6]:
import re
import unicodedata
import pandas as pd

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_text(text):
	text = unicode_to_ascii(text.lower().strip())
	# convert ... and .. to .
	text = re.sub(r"\.\.\.", ". ", text)
	text = re.sub(r"\.\.", ". ", text)
	
	# Abbreviation Restoration & Stem Preservation
	text = re.sub(r"i'm", "i am", text)
	text = re.sub(r"he's", "he is", text)
	text = re.sub(r"she's", "she is", text)
	text = re.sub(r"it's", "it is", text)
	text = re.sub(r"that's", "that is", text)
	text = re.sub(r"what's", "what is", text)
	text = re.sub(r"where's", "where is", text)
	text = re.sub(r"how's", "how is", text)
	text = re.sub(r"\'ll", " will", text)
	text = re.sub(r"\'ve", " have", text)
	text = re.sub(r"\'re", " are", text)
	text = re.sub(r"\'d", " would", text)
	text = re.sub(r"won't", "will not", text)
	text = re.sub(r"can't", "cannot", text)
	text = re.sub(r"n't", " not", text)

	# delete most punctuation marks
	text = re.sub(r"[-\"#/@;:{}`+=~|]", "", text)

	# Multiple spaces merge
	text = re.sub(r"\s+", " ", text).strip()
	return text

def clean_Punctuation(text):
	# clean text by removing punctuation and special characters
	text = re.sub(r'[^\w\s]', '', text)
	return text

def get_data(csv_path, loadDataSize):
	df = pd.read_csv(csv_path).dropna()
	print(df.columns)
	per_class_count = int(loadDataSize / 5)
	selected_dfs = []

	for category in df['File_path'].unique():
		category_df = df[df['File_path'] == category].head(per_class_count)
		selected_dfs.append(category_df)

	selected_df = pd.concat(selected_dfs).sample(frac=1).reset_index(drop=True)  # shuffle the selected data
	
	original_content = selected_df['Articles']
	
	# 提取每一条selected_df里Articles中".."前面的内容，作为title添加到selected_df
	selected_df['title'] = selected_df['Articles'].apply(lambda x: x.split('..', 1)[0] if '..' in x else x)

	# 去掉articles中第一个".."前面的内容，将剩下内容作为content添加到selected_df
	selected_df['Articles'] = selected_df['Articles'].apply(lambda x: x.split('..', 1)[1] if '..' in x else '')

	# 清洗selected_df里的每一条数据，使用clean_text函数
	selected_df['Articles'] = selected_df['Articles'].apply(clean_text)
	selected_df['title'] = selected_df['title'].apply(clean_text)
	selected_df['Summaries'] = selected_df['Summaries'].apply(clean_text)
	# # add data without punctuation, this is for self training models
	# selected_df['Articles_no_punctuation'] = selected_df['Articles'].apply(clean_Punctuation)
	# selected_df['title_no_punctuation'] = selected_df['title'].apply(clean_Punctuation)
	# selected_df['Summaries_no_punctuation'] = selected_df['Summaries'].apply(clean_Punctuation)
	

	# 返回提取到的数据，并将其转换为列表
	data = selected_df.values.tolist()  # 0:type, 1:content, 2:summary, 3:title
	return data




In [7]:
testDataSet = get_data("data/bbc-news-summary.csv", 20)
firstData = testDataSet[0]
print(f"firstData type: {firstData[0]}")
print(f"firstData content: {firstData[1]}")
print(f"firstData summary: {firstData[2]}")
print(f"firstData title: {firstData[3]}")


Index(['File_path', 'Articles', 'Summaries'], dtype='object')
firstData type: entertainment
firstData content: clint eastwood's million dollar baby beat martin scorsese's the aviator to the top awards at the oscars on sunday. the boxing drama was named best picture and eastwood pipped scorsese to best director, while its stars hilary swank and morgan freeman won acting awards. the aviator took five prizes including best supporting actress for cate blanchett. the biopic of howard hughes led the nominations with 11. jamie foxx picked up best actor for playing soul star ray charles in ray. many expected this to be scorsese's year but he was unsuccessful in the best director category for the fifth time in his career. he has never won despite being nominated for such films as raging bull and goodfellas. instead, eastwood took his second oscar for best director after winning for unforgiven in 1993. it was a wonderful adventure, eastwood said. to make a picture in 37 days, it takes a well oil

In [8]:
inputtext = "russian authorities do not have to abide by any us court decisions taken with regard to troubled oil giant yukos, a houston court has been told. legal expert william butler said there was no treaty between the us and russia to recognise the other's legal rulings. that meant moscow would not have to adhere to us rulings in the yukos case. yukos says a us court was entitled to declare it bankrupt before its yugansk unit was sold, since it has a us subsidiary and local bank accounts. yukos made its surprise chapter 11 bankruptcy filing in houston in december in an unsuccessful attempt to halt the auction of yugansk, its main oil producing unit, by russian authorities. yugansk was sold to help pay off a $27.5bn (£14.5bn) back tax bill. it was bought for $9.4bn by a previously unknown group, which was in turn bought by statecontrolled oil company rosneft. the us court's jurisdiction has been challenged by deutsche bank and gazpromneft, a former unit of russian gas monopoly gazprom which is due to merge with rosneft. deutsche bank maintains the case has no place in a us court because yukos has no assets in the us, apart from two bank accounts and a house in houston owned by its chief finance officer bruce misamore. deutsche bank is involved in the case because it is itself being sued by yukos. it had agreed to loan gazpromneft the money to bid for yugansk. us bankruptcy judge letitia clark, who issued an injunction in december to try and prevent the yugansk sale, has said she will rule pretty promptly, however i do not anticipate ruling on it before next tuesday. yukos has claimed it sought help in the us because other forums russian courts and the european court of human rights were either unfriendly or offered less protection. it has claimed that russia imposed the huge tax bill and forced the sale of yugansk as part of a campaign to destroy yukos and its former owner mihkail khodorkovsky, who is facing a 10year prison term in russia for fraud and tax evasion. yukos' parent company, the gibraltarbased menatep group, is suing russia in europe for $28.3bn in financial damages. the company is also seeking $20bn in a separate us lawsuit against rosneft and gazprom for their role in the sale of yugansk."

inputTitle = "yukos case: russia not bound by us court ruling"

chatBot=ChatBot()
chatBot.set_chosen_model("GRU")
chatBot.set_model_weight("GRU", "train/checkpoint/GRU_seq2seq_bbc_500_fixed.h5")

summary = chatBot.news_summary(inputTitle=inputTitle, inputtext=inputtext)

print(f"summary: {summary}")

summary: this is $27 $27 should be $27 by the $27 union said it is $27 $27 was $27 in the us dollar will be more than three years the only have had in the current three years had been $27 more than three years to be the only have the current month already had a more than three more than three years to be a $27 and more than three years to be the only have had the current month already had a more than three more than three years to be a $27 and more than three years to be the only have had the current month already had a more than three more than three years to be a $27 and more than three years to be the only have had the current month already had a more than three of the current month already had a more than three of the current month already had a


In [9]:
# 计算 BERTScore
P, R, F1 = bert_score(["i love machine learning  "], ["i like artificial intelligence"], lang='en',model_type='roberta-large', verbose=False)

# 打印每一对的分数
for i in range(len(F1)):
    print(f"[Sample {i+1}] Precision: {P[i]:.4f}, Recall: {R[i]:.4f}, F1: {F1[i]:.4f}")

[Sample 1] Precision: 0.9534, Recall: 0.9534, F1: 0.9534


In [10]:
import time

def clean_Punctuation(text):
	# clean text by removing punctuation and special characters
	text = re.sub(r'[^\w\s]', '', text)
	return text

def modelEvaluation(chatBot, testDataSet, model, model_weight):
	# load the model
	# chatBot=ChatBot()
	chatBot.set_chosen_model(model)
	chatBot.set_model_weight(model, model_weight)

	# calculating average ROUGE scores, BERT score and total responses time
	scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
	rouge1_f1, rouge2_f1, rougel_f1 = [], [], []
	bert_f1 = []
	total_response_time = 0

	for data in testDataSet:
		if model == "T5" or model == "textRank":
			# pretrained model that not need for punctuation cleaning
			# including T5
			original_summary = data[2] 
			original_title = data[3]
			original_content = data[1]
		else:
			# pretrained model that need for punctuation cleaning
			# including LSTM
			original_summary = clean_Punctuation(data[2])
			original_title = clean_Punctuation(data[3])
			original_content = clean_Punctuation(data[1])

		startTime = time.time()

		summary = chatBot.news_summary(inputTitle=original_title, inputtext=original_content)

		endTime = time.time()
		response_time = endTime - startTime
		total_response_time += response_time

		# ROUGE score
		rouge_scores = scorer.score(original_summary, summary)
		rouge1_f1.append(rouge_scores['rouge1'].fmeasure)
		rouge2_f1.append(rouge_scores['rouge2'].fmeasure)
		rougel_f1.append(rouge_scores['rougeL'].fmeasure)
		# BERT score
		P, R, F1 = bert_score([original_summary], [summary], lang='en')
		bert_f1.append(F1.mean().item())

	avg_time = total_response_time * 1000 // len(testDataSet)

	# print average rouge score
	print(f"======Average ROUGE Scores======")
	print(f"==>model: {chatBot.get_chosen_model()}")
	print(f"==>model weight: {chatBot.get_model_weight()}")
	print(f"ROUGE-1 F1: {sum(rouge1_f1) / len(rouge1_f1):.4f}")
	print(f"ROUGE-2 F1: {sum(rouge2_f1) / len(rouge2_f1):.4f}")
	print(f"ROUGE-L F1: {sum(rougel_f1) / len(rougel_f1):.4f}")
	print(f"Average BERT F1: {sum(bert_f1) / len(bert_f1):.4f}")
	print(f"Total response time for {len(testDataSet)} test inputs: {avg_time:.4f} millisecond")
	
	return total_response_time, sum(rouge1_f1) / len(rouge1_f1), sum(rouge2_f1) / len(rouge2_f1), sum(rougel_f1) / len(rougel_f1), sum(bert_f1) / len(bert_f1)

In [ ]:

allDict = []
allDict.append(modelEvaluation(CHATBOT, testDataSet, "LSTM", "train/checkpoint/seq2seq_bbc_500_fixed.pt"))
allDict.append(modelEvaluation(CHATBOT, testDataSet, "LSTM", "train/checkpoint/seq2seq_bbc_1500_fixed.pt"))
allDict.append(modelEvaluation(CHATBOT, testDataSet, "GRU", "train/checkpoint/GRU_seq2seq_bbc_500_fixed.h5"))
allDict.append(modelEvaluation(CHATBOT, testDataSet, "GRU", "train/checkpoint/GRU_seq2seq_bbc_1500_fixed.h5"))
allDict.append(modelEvaluation(CHATBOT, testDataSet, "T5", "N/A"))
allDict.append(modelEvaluation(CHATBOT, testDataSet, "textRank", "N/A"))



======Average ROUGE Scores======
==>model: LSTM
==>model weight: train/checkpoint/seq2seq_bbc_500_fixed.pt
ROUGE-1 F1: 0.2263
ROUGE-2 F1: 0.0229
ROUGE-L F1: 0.1315
Average BERT F1: 0.7532
Total response time for 20 test inputs: 123.0000 millisecond
======Average ROUGE Scores======
==>model: LSTM
==>model weight: train/checkpoint/seq2seq_bbc_1500_fixed.pt
ROUGE-1 F1: 0.2369
ROUGE-2 F1: 0.0286
ROUGE-L F1: 0.1300
Average BERT F1: 0.7601
Total response time for 20 test inputs: 868.0000 millisecond
======Average ROUGE Scores======
==>model: GRU
==>model weight: train/checkpoint/GRU_seq2seq_bbc_500_fixed.h5
ROUGE-1 F1: 0.1374
ROUGE-2 F1: 0.0276
ROUGE-L F1: 0.1057
Average BERT F1: 0.7414
Total response time for 20 test inputs: 3070.0000 millisecond
======Average ROUGE Scores======
==>model: GRU
==>model weight: train/checkpoint/GRU_seq2seq_bbc_1500_fixed.h5
ROUGE-1 F1: 0.1385
ROUGE-2 F1: 0.0280
ROUGE-L F1: 0.1063
Average BERT F1: 0.7466
Total response time for 20 test inputs: 3244.0000 millis